In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet import ResNet50
from keras.applications.resnet import ResNet101
from keras.applications.resnet import ResNet152
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd gdrive/

/content/gdrive


In [ ]:
%cd MyDrive/

/content/gdrive/MyDrive


In [ ]:
img_size = 128

train_path = 'trainSet'
#test_path = 'testSet'

folders = glob('trainSet/*')
print(len(folders))

import pathlib,PIL
data_dir = pathlib.Path('/content/gdrive/MyDrive/trainSet')
print(data_dir)
#ears = list(data_dir.glob('077/*'))
#print(ears)
#PIL.Image.open(str(ears[0]))

100
/content/gdrive/MyDrive/trainSet


In [ ]:
# add preprocessing layer to the front of VGG
#vgg = VGG16(input_shape=(128,128,3), weights='imagenet', include_top=False)
#resnet50 = ResNet50(input_shape=(128,128,3), weights='imagenet', include_top=False)
#model = Sequential()
model = Sequential([
    layers.RandomFlip("horizontal", input_shape=(img_size, img_size, 3)),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
  ])

# change ResNet50, ResNet101, ResNet152, VGG16
pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(img_size,img_size,3),
                   pooling='max',
                   classes=100,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

model.add(pretrained_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(100, activation='softmax'))

In [ ]:
# Apply data augmentation
#data_augmentation = keras.Sequential(
#  [
#    layers.RandomFlip("horizontal", input_shape=(img_size, img_size, 3)),
#    layers.RandomRotation(0.1),
#    layers.RandomZoom(0.1),
#  ]
#)

In [ ]:
# don't train existing weights
#for layer in vgg.layers:
#  layer.trainable = False
#for layer in resnet50.layers:
#  layer.trainable = False

In [ ]:
#x = Flatten()(vgg.output)
#x = Flatten()(resnet50.output)

#prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
#model = Model(inputs=vgg.input, outputs=prediction)
#model = Model(inputs=resnet50.input, outputs=prediction)

# view the structure of the model
model.summary()


In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
batch_size=32
training_set = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="training",
  seed=123,
  image_size=(img_size, img_size),
  batch_size=batch_size)

Found 750 files belonging to 100 classes.
Using 675 files for training.


In [ ]:
test_set = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.1,
  subset="validation",
  seed=123,
  image_size=(img_size, img_size),
  batch_size=batch_size)

Found 750 files belonging to 100 classes.
Using 75 files for validation.


In [ ]:
#from keras.preprocessing.image import ImageDataGenerator

#train_datagen = ImageDataGenerator(rescale = 1./255,
#                                   shear_range = 0.1,
#                                   zoom_range = 0.1,
#                                   horizontal_flip = True,
#                                   vertical_flip = True)

#test_datagen = ImageDataGenerator(rescale = 1./255)

# Generating the Training Data
#training_set = train_datagen.flow_from_directory(
#        train_path,
#        target_size=(64, 64),
#        batch_size=32,
#        class_mode='categorical')
 
 
# Generating the Testing Data
#test_set = test_datagen.flow_from_directory(
#        train_path,
#        target_size=(64, 64),
#        batch_size=32,
#        class_mode='categorical')

In [ ]:
#print(test_set.class_indices)

In [ ]:
#from keras import callbacks
#earlystopping = callbacks.EarlyStopping(monitor ="val_loss", 
#                                        mode ="min", patience = 20, 
#                                        restore_best_weights = True)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = "modelCheckpoint/checkpoint-{epoch:02d}-{val_accuracy:.2f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

In [ ]:
# fit the model
#r = model.fit(
#  training_set,
#  validation_data=test_set,
#  epochs=50,
#  steps_per_epoch=len(training_set),
#  validation_steps=len(test_set),
#  callbacks =[checkpoint]
#)
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=30,
  callbacks =[checkpoint]
)

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

In [ ]:
from keras.models import load_model

#model.save('vgg_best_model.h5')
model.save('resnet50_best_model.h5')

In [ ]:
#https://towardsdatascience.com/step-by-step-vgg16-implementation-in-keras-for-beginners-a833c686ae6c

from keras.preprocessing import image
img = image.load_img("testSet/001/0004.png",target_size=(img_size,img_size))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)

#from keras.models import load_model
#saved_model = load_model("resnet50_best_model.h5")

output = model.predict(img)
print(output)

pred = np.argmax(output) + 1
print(pred)